In [11]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [12]:
# Load all the train model, scaler, encoder
model=load_model('model.h5')

with open('label_encode_gender.pkl','rb') as file:
    label_gender=pickle.load(file)

with open('encode_geography.pkl','rb') as file:
    encode_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [13]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 32,
    'Tenure': 2,
    'Balance': 75000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 120000
}

In [14]:
# One-hot encoding for Geography
geo_encoded = encode_geo.transform ([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=encode_geo.get_feature_names_out(['Geography']))

c:\Users\iamsh\Python\envs\denv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [15]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [16]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Female,32,2,75000,2,1,1,120000


In [17]:
input_df['Gender']

0    Female
Name: Gender, dtype: object

In [18]:
# Encode other categorical variables
input_df['Gender']= label_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,32,2,75000,2,1,1,120000


In [19]:
# Combine one-hot encoded columns with input data
input_df= pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,32,2,75000,2,1,1,120000,1.0,0.0,0.0


In [21]:
# Scaling the input data
input_scaled= scaler.transform(input_df)
input_scaled

array([[-0.53598516, -1.09499335, -0.6557859 , -1.04241787, -0.01764635,
         0.80843615,  0.64920267,  0.97481699,  0.34023471,  1.00150113,
        -0.57946723, -0.57638802]])

In [22]:
# Predict the churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 230ms/step


array([[0.00799478]], dtype=float32)

In [24]:
prediction_proba= prediction[0][0]
prediction_proba

0.007994776

In [25]:
if prediction_proba > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
